In [ ]:
# =========================
# 📚 PDF Q&A RAG — Launcher
# =========================

# 1) Install deps
!pip -q install streamlit langchain-community faiss-cpu sentence-transformers \
                transformers accelerate safetensors pypdf > /dev/null

# 2) Write the Streamlit app
app_code = r"""
import os
import io
import torch
import streamlit as st

# ---- LangChain & friends ----
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ---- PDF parsing ----
from pypdf import PdfReader

# ---- Local LLM (FLAN-T5) ----
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

st.set_page_config(page_title="PDF Q&A (Local RAG)", page_icon="📚")
st.title(" PDF Q&A Chatbot — Local RAG (LangChain + FLAN-T5)")

st.markdown(
    "Upload one or more PDFs. We’ll chunk + embed them (MiniLM), build a FAISS index, "
    "then answer questions using retrieved chunks and a local FLAN-T5 model."
)

# ---------------------------
# CACHED MODELS
# ---------------------------
@st.cache_resource
def load_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    return HuggingFaceEmbeddings(model_name=model_name)

@st.cache_resource
def load_flan():
    name = "google/flan-t5-base"
    tok = AutoTokenizer.from_pretrained(name)
    dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    model = AutoModelForSeq2SeqLM.from_pretrained(name, torch_dtype=dtype)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    return tok, model, device

embeddings = load_embeddings()
tokenizer, flan, device = load_flan()

# ---------------------------
# HELPERS
# ---------------------------
def read_pdfs(files):
    texts = []
    for f in files:
        data = f.read()
        reader = PdfReader(io.BytesIO(data))
        content = []
        for page in reader.pages:
            try:
                content.append(page.extract_text() or "")
            except Exception:
                content.append("")
        full_text = "\\n".join(content).strip()
        if full_text:
            texts.append(full_text)
    return texts

def build_vectorstore(raw_texts):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=150,
        length_function=len,
    )
    docs = []
    for t in raw_texts:
        docs.extend(splitter.create_documents([t]))
    vs = FAISS.from_documents(docs, embeddings)
    return vs

def make_prompt(question, contexts):
    context_block = "\\n\\n".join([f"[Chunk {i+1}]\\n{c}" for i, c in enumerate(contexts)])
    prompt = (
        "You are a helpful assistant. Answer the question using ONLY the context.\\n"
        "If the answer is not in the context, say you don't know.\\n\\n"
        f"Context:\\n{context_block}\\n\\n"
        f"Question: {question}\\nAnswer:"
    )
    return prompt

def generate_answer(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        output = flan.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.2,
            do_sample=False,
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# ---------------------------
# UI
# ---------------------------
st.subheader("📤 Upload PDFs")
uploaded = st.file_uploader("Upload one or more PDFs", type=["pdf"], accept_multiple_files=True)

if 'vectorstore' not in st.session_state:
    st.session_state.vectorstore = None

col_a, col_b = st.columns([1,1])
with col_a:
    build_btn = st.button("🔧 Build / Rebuild Index")
with col_b:
    clear_btn = st.button("🗑️ Clear Index")

if clear_btn:
    st.session_state.vectorstore = None
    st.success("Cleared vector index.")

if build_btn:
    if not uploaded:
        st.warning("Please upload at least one PDF.")
    else:
        with st.spinner("Reading PDFs and building FAISS index..."):
            texts = read_pdfs(uploaded)
            if not any(texts):
                st.error("No extractable text found in the PDFs.")
            else:
                st.session_state.vectorstore = build_vectorstore(texts)
                st.success("Index ready! Ask questions below.")

st.divider()
st.subheader(" Ask a Question")

q = st.text_input("Your question")
k = st.slider("Top-k chunks", 2, 8, 4)
max_tokens = st.slider("Max new tokens (answer length)", 64, 512, 256, step=32)

if st.session_state.vectorstore is None:
    st.info(" Upload PDFs and click **Build / Rebuild Index** to start.")
else:
    if st.button(" Retrieve & Answer", disabled=(not q.strip())):
        with st.spinner("Retrieving relevant chunks..."):
            docs = st.session_state.vectorstore.similarity_search(q, k=k)
            contexts = [d.page_content for d in docs]
        st.write("**Retrieved Chunks:**")
        for i, c in enumerate(contexts, 1):
            with st.expander(f"Chunk {i}"):
                st.write(c)

        with st.spinner("Generating answer with FLAN-T5..."):
            prompt = make_prompt(q, contexts)
            ans = generate_answer(prompt, max_new_tokens=max_tokens)
        st.success("**Answer:**")
        st.write(ans)
"""

with open("app.py", "w") as f:
    f.write(app_code)

# 3) Kill anything on 8501 and start Streamlit
!pkill -f streamlit || true
!fuser -k 8501/tcp || true
!streamlit run app.py --server.port 8501 --server.headless true &>/dev/null&

# 4) Open a Cloudflare tunnel (no account required) and print the URL
import subprocess, time, re, sys

# fetch latest cloudflared
!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared

p = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:8501", "--no-autoupdate"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

url = None
for _ in range(300):  # ~30s
    line = p.stdout.readline()
    if not line:
        time.sleep(0.1); continue
    if "trycloudflare.com" in line:
        for token in line.split():
            if token.startswith("https://") and "trycloudflare.com" in token:
                url = token.strip()
                break
    if url:
        break

print(" Streamlit app URL:", url if url else "Still starting... check the logs above.")


^C
✅ Streamlit app URL: https://stands-happens-devoted-vitamin.trycloudflare.com


In [ ]:
!pip install streamlit langchain-community faiss-cpu sentence-transformers \
            transformers accelerate safetensors pypdf -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 725.1 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
